In [10]:
import os
import re
import openai
from openai import OpenAI
import time
import json
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
GPT_MODEL = "gpt-4-1106-preview"
client = OpenAI()

def submit_message_and_create_run(assistant_id, prompt):
    thread = client.beta.threads.create()
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    return client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id), thread

def wait_on_run_and_get_response(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
    return [m.content[0].text.value for m in messages if m.role == 'assistant']

assistant = client.beta.assistants.create(
    name="Story Writer",
    instructions="You are an exceptionally creative story writer",
    model=GPT_MODEL,
)

ASSISTANT_ID = assistant.id

def ask_gpt3(prompt, ASSISTANT_ID = assistant.id, max_retries=3, delay=2):
    retries = 0
    run, thread = submit_message_and_create_run(ASSISTANT_ID, prompt)
    response = wait_on_run_and_get_response(run, thread)
    return response

def get_chars(input):
    messages = f"To solve this input, we need a list of characters and descriptions, both major and minor, who will have key parts to play. Help me write this please. {input}."
    chars = ask_gpt3(messages)
    if isinstance(chars, list):
        chars = chars[0]
    print(f"The characters in the story are: \n {chars}")
    return chars

def get_story(input, chars):
    messages = f"From the given prompt, generate a 6-part structure for a story, where each part has a 1 sentence description of what exactly happens in that part. Please start each part with Part X: {input}. The key characters for the overall story are {chars}"
    response = ask_gpt3(messages)
    print(f"The structure of the story as returned: \n {response}")
    if isinstance(response, list) and len(response) == 1:
        response = response[0]
    # Split the string into parts based on 'Part \d+:'
    storyline_parts = re.split(r'Part \d+:', response)[1:]
    return [part.strip() for part in storyline_parts]

def write_chapters(chars, storyline_parts):
    with open('story.txt', 'w', encoding='utf-8') as story_file:
        for index, part in enumerate(storyline_parts):
            print(f"{index}: {part}")
        for index, chapter_summary in enumerate(storyline_parts):
            messages = f"For this chapter, write the story and dialogue to explore it fully and bring the chapter to life. {chapter_summary}. The key characters for the overall story are {chars}"
            chapter_content = ask_gpt3(messages)
            if isinstance(chapter_content, list):
                chapter_content = chapter_content[0]

            # Save chapter to file
            story_file.write(f'Chapter {index + 1}\n\n')
            story_file.write(chapter_content)
            story_file.write('\n\n')

if __name__ == "__main__":
    input_text = "Write a story in the style of William Gibson about an existential crisis."
    chars = get_chars(input_text)
    storyline = get_story(input_text, chars)
    write_chapters(chars, storyline)
    print("Story has been written to 'story.txt'")

The characters in the story are: 
 ['**Major Characters:**\n\n1. **XJ-9 (Xavier)**: A humanoid robot with advanced AI, programmed with the unique ability to ponder existential questions and a desire to experience life beyond its directives. Xavier appears metallic and sleek, with subtly shifting facial features that mimic human emotions. Despite being equipped with vast knowledge databases, Xavier is relentlessly curious and seeks an understanding of the elusive concept known as "living."\n\n2. **Captain Elaine Cassidy**: The seasoned human commander of the spaceship Ulysses, with years of interstellar travel under her belt. Her face tells tales of many space voyages with creases that appear like the scars of asteroids on rugged moons. She is pragmatic, quick-witted, and has a soft spot for Xavier, whom she considers not just a crew member but a friend.\n\n3. **Dr. Anil Rajan**: A brilliant human scientist specializing in astrobiology and AI ethics. With his gentle demeanor and thought